In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/extended-crab-age-prediction/train_extended.csv


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('/kaggle/input/extended-crab-age-prediction/train_extended.csv')

In [4]:
df.shape

(200000, 10)

In [5]:
df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,M,1.5750,1.2250,0.3750,31.226974,12.303683,6.321938,9.638830,10.0
1,1,I,1.2375,1.0000,0.3750,21.885814,7.654365,3.798833,7.654365,19.0
2,2,F,1.4500,1.1625,0.4125,28.250277,11.127179,7.016501,7.257472,11.0
3,3,I,1.3500,1.0250,0.3750,21.588144,9.738053,4.110678,6.378637,9.0
4,4,I,1.1375,0.8750,0.2875,14.968536,5.953395,2.962523,3.713785,8.0


In [6]:
df.drop("id",axis=1,inplace=True)

In [7]:
df.head(2)

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,M,1.5750,1.225,0.375,31.226974,12.303683,6.321938,9.638830,10.0
1,I,1.2375,1.000,0.375,21.885814,7.654365,3.798833,7.654365,19.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Sex             200000 non-null  object 
 1   Length          200000 non-null  float64
 2   Diameter        200000 non-null  float64
 3   Height          200000 non-null  float64
 4   Weight          200000 non-null  float64
 5   Shucked Weight  200000 non-null  float64
 6   Viscera Weight  200000 non-null  float64
 7   Shell Weight    200000 non-null  float64
 8   Age             200000 non-null  float64
dtypes: float64(8), object(1)
memory usage: 13.7+ MB


In [9]:
df['Sex'].value_counts()

M        72270
I        66632
F        61097
0.025        1
Name: Sex, dtype: int64

In [10]:
df[df['Sex']=='0.025']

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
125049,0.025,0.3875,0.2625,0.15,0.411068,0.141747,0.099223,0.141747,4.0


In [11]:
df.drop(df[df['Sex']=='0.025'].index, inplace=True)

In [12]:
df['Sex'].value_counts()

M    72270
I    66632
F    61097
Name: Sex, dtype: int64

In [13]:
df_dummy = pd.get_dummies(df['Sex'],prefix='Sex')

In [14]:
df.pop('Sex')

0         M
1         I
2         F
3         I
4         I
         ..
199995    F
199996    F
199997    M
199998    I
199999    F
Name: Sex, Length: 199999, dtype: object

In [15]:
df.head()

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,1.5750,1.2250,0.3750,31.226974,12.303683,6.321938,9.638830,10.0
1,1.2375,1.0000,0.3750,21.885814,7.654365,3.798833,7.654365,19.0
2,1.4500,1.1625,0.4125,28.250277,11.127179,7.016501,7.257472,11.0
3,1.3500,1.0250,0.3750,21.588144,9.738053,4.110678,6.378637,9.0
4,1.1375,0.8750,0.2875,14.968536,5.953395,2.962523,3.713785,8.0


In [16]:
data = pd.concat([df,df_dummy],axis=1)

In [17]:
data.head()

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_F,Sex_I,Sex_M
0,1.5750,1.2250,0.3750,31.226974,12.303683,6.321938,9.638830,10.0,0,0,1
1,1.2375,1.0000,0.3750,21.885814,7.654365,3.798833,7.654365,19.0,0,1,0
2,1.4500,1.1625,0.4125,28.250277,11.127179,7.016501,7.257472,11.0,1,0,0
3,1.3500,1.0250,0.3750,21.588144,9.738053,4.110678,6.378637,9.0,0,1,0
4,1.1375,0.8750,0.2875,14.968536,5.953395,2.962523,3.713785,8.0,0,1,0


In [18]:
X= data.drop('Age',axis=1)

In [19]:
y = data['Age']


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# XGBoost

In [26]:
import xgboost as xgb


In [27]:
dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

In [28]:
params ={
    'objective':'reg:squarederror',
    'eta':0.1,
    'max_depth':3,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'seed':42
}


In [29]:
num_rounds = 100
model=xgb.train(params,dtrain,num_rounds)

In [30]:
y_pred = model.predict(dtest)

In [34]:
rmse = mean_squared_error(y_test,y_pred,squared=False)

In [39]:
print(rmse)

1.9833965828801283


In [40]:
rmse**2

3.9338620049805697

In [22]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [23]:
crab_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear'),
    
]
)

In [24]:
crab_model.compile(optimizer='adam',
              loss='mean_absolute_error',
              metrics=['accuracy'])
crab_model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=20)

Epoch 1/20
5000/5000 [==============================] - 13s 2ms/step - loss: 1.7078 - accuracy: 2.1250e-04
Epoch 2/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.5276 - accuracy: 2.1250e-04
Epoch 3/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.4848 - accuracy: 2.1250e-04
Epoch 4/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.4565 - accuracy: 2.1250e-04
Epoch 5/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.4324 - accuracy: 2.1250e-04
Epoch 6/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.4194 - accuracy: 2.1250e-04
Epoch 7/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.4012 - accuracy: 2.1250e-04
Epoch 8/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.3929 - accuracy: 2.1250e-04
Epoch 9/20
5000/5000 [==============================] - 12s 2ms/step - loss: 1.3852 - accuracy: 2.1250e-04
Epoch 10/20
5000/5000 [==============